In [61]:
#This implementation for Locally Weighted Regression aims to demonstrate its application for estimating values of actual Non-Linear graphs for a given query.
#The algorithm uses the concept of Gaussian weights, which determine the "importance" or "weight" of any given C_i due to the ith sample. 
#This helps in approximating a linear function that is approximately equivalent to the true graph around a small neighbourhood of the query point.
#The process is similar to computing normal equations for the analytical solution of theta in gradient descent with linear regression.

# Let's start:

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
#Importing all the basic libraries needed for operations.

In [63]:
X = np.random.uniform(-10 * np.pi, 10 * np.pi, size = (2000, 10))
Y = np.sum(np.sin(X), axis = 1) + 0.3 * np.sum(X**2, axis = 1) + 2 * np.random.randn(2000)
#We will synthesise a dataset matrix X with 2000 samples (rows) and 10 features (columns).
#Each entry of this matrix will range uniformly from [-10*pi, 10*pi]. 
#We also synthesise a random base function off of which our Y matrix will be centered.
#To simulate approximate real-world conditions, we introduce gaussian noise to this as well.

## Notes about the function choice:
1. Our true function is randomly defined such that $\vec{X} \in \mathbb{R}^{n}$ , where: $F_{true} (\vec{X}) = \sum_{j = 1}^{n} sin(x_{j}) + 0.3 \sum_{j = 1}^{n} x_{j}^{2}$ where $x_{j}$ is the $j^{th}$ co-ordinate/entry of $\vec{X}$ input row vector.
2. Now, to simulate real world conditions, we introduce a gaussian noise to the true function, such that: $Y = F_{true} (\vec{X}) + 2\eta$ where $\eta $ ~ $\mathcal{N} (\mu = 0, \sigma^{2} = 1)$ , where Y is our target matrix.
3. The purpose of making a non-linear function is satisfied here, along with simulating real world conditions.
4. The constant and functions so chosen in $F_{true} (\vec{X})$, the distribution of entries of X matrix being uniform are all design choices.
5. However, the noise being distributed under a Gaussian PMF is a much needed condition for LWR to hold.

In [64]:
one = np.ones((2000, 1))
X = np.hstack([one, X])
Y = Y.reshape(-1, 1)
#Adding the bias term to account for the noise as accurately as possible.
print(X.shape)
print(Y.shape)
#Verifying the shapes.

(2000, 11)
(2000, 1)


In [65]:
X_query = X[::100]
Y_verify = Y[::100]
#We defined every 100 entries in the X matrix to function as our query matrix. This is a design choice.
#This will lead to a total of 20 sample queries in our query space, whose predicted values can be verified from the Y matrix as well.
print(X_query.shape)
print(Y_verify.shape)
#Verifying the query space shape.

(20, 11)
(20, 1)


In [66]:
q = 0
#q is the query iterator.
tau = 1
#Assuming a standard deviation of 1 along the gaussian weights. Arbitrarily chosen.
Y_preds = []
#An array/list to store the predicted values.
while(q < 20):
    X_q = X_query[[q], :]
    #Fetching the X_q query vector as a row.
    weights_q = np.exp(-np.sum((X - X_q)**2, axis = 1, keepdims = True) / (2 * tau**2))
    #Making a column vector of weights. Instead of using direct matrix multiplication, we will use vectorized multiplication in our methods.
    X_weighted = X * weights_q
    Y_weighted = Y * weights_q
    #These two are essential components that the inverse and the usual term in the closed form formula comprise of.
    Q = np.linalg.pinv(X.T @ X_weighted) @ (X.T @ Y_weighted)
    #The above 4 statements demonstrate an easier method to calculate the close form solution for Theta, without the need of a W matrix.
    #This formula yields the same result as the general closed-form solution.
    print(X_q @ Q)
    print(Y[q, 0])
    #Manually make comparisons between the predicted and actual values for the query. 
    Y_preds.append(X_q @ Q)
    #Record the predicted values per query.
    q += 1

[[1034.83638286]]
1034.836382859744
[[693.87624651]]
961.9687332346251
[[1145.96585291]]
786.7616819608722
[[1036.97734137]]
702.086632319787
[[725.27894244]]
1105.6598426702196
[[1293.94132898]]
983.3773784247517
[[409.1240999]]
1036.4219147651527
[[797.38110532]]
591.8716023021316
[[666.70064946]]
818.6951240628097
[[956.01522303]]
598.0437458889023
[[1230.62628786]]
849.858598158439
[[1885.24798861]]
930.3466800847214
[[1167.60889969]]
1092.1756762599293
[[858.59751177]]
398.44230322080625
[[821.71045268]]
682.4890249265949
[[377.4045915]]
821.8556131198612
[[410.24378108]]
1383.215765647978
[[681.04603873]]
635.9669528481256
[[1020.85345562]]
334.52408324840536
[[482.94046779]]
695.4770178383808


In [67]:
print(mean_squared_error(Y_verify, np.array(Y_preds).reshape(-1, 1)))
#Checking our mean-squared error between the actual and predicted values for the query points. 
#The closer this value is to 0, the more the accuracy.
#Y_preds had earlier been defined as a simple python array. To reshape it into a matrix, we need to broadcast it into a numpy array.

6.268478079503412e-26


## Notes:
1. As we can see, the value for our mean squared error, $MSE = \sum_{q = 1}^{20} \frac{Y_{real}^{q} - Y_{predicted}^{q}}{20}$ is close to 0 (in the order of $10^{-26}$ ).
2. This yields from the analytical closed form solution for our parameter vector $\vec{\theta^{*}}$ such that $\theta^{q} = (X^{T}WX)^{-1} (X^{T}WY)$ where W is the square diagonal matrix of the shape $m$ x $m$, such that the $i^{th}$ row contains the gaussian weight $\omega^{i}_{q}$ for the $q^{th}$ query at position $[W_{ii}]$.
3. Here, the gaussian weight $\omega^{i}_{q} = exp(-\frac{||X^{i} - X^{q}||_{2}^{2}}{2 \tau^{2}})$ where $X^{i}$ is the $i^{th}$ sample and the $X^{q}$ is the $q^{th}$ query such that $X^{i}, X^{q} \in \mathbb{R}^{n}$.
4. Now, we have used a simplified formula for this: $\vec{\theta^{*}} = (X^{T}X_{weighted})^{-1} (X^{T}Y_{weighted})$, where $X_{weighted} = X * weights$ and $Y_{weighted} = Y * weights$, where "weights" is the column vector of shape $m$ x $1$, with $i^{th}$ element given by $\omega^{i}_{q}$.
5. The formula might not make a difference mathematically. But measuring in terms of time complexity, the original closed form solution has $O(nm^{2})$ time complexity, whereas our improved form has complexity $O(mn^{2})$. Considering the fact that we have set (no. of features n) < (no. of samples m), vectorized operations in our improved form significantly improve computation speed for m >> 1.